first we import all the neccessary packages for working with the data

In [3]:
import os
from IPython.display import display
#import hoboreader
import pandas as pd
import shutil
import matplotlib as mp
import re

next we ensure that our working directory is set correctly so that when we're working with file paths they always work relative to our project

In [4]:
#os.chdir("..")
os.chdir("C:\\Users\\samer\\OneDrive - Cal Poly\\Classes\\SPR_Fellowship\\SPR_Fellowship")
os.getcwd()

'C:\\Users\\samer\\OneDrive - Cal Poly\\Classes\\SPR_Fellowship\\SPR_Fellowship'

First well create a function for searching through complex fill structures and returning the paths to files of only a certain file type

In [5]:
def search_filetype(path = ".", extentension = ""):
    """
    :param path: string, where to search
    :param extentension: string, what files to look for
    :return: list, paths to all files with a matching extension
    """
    paths = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename[-len(extentension):] == extentension:
                paths.append(dirpath + "/" + filename)
    return paths

Well also want some test data to work with along the way before we start working with thousands of files

In [6]:
list_ex_headers = search_filetype(path="LegacyFiles/All_txt/txt_header_examples/", extentension=".txt")

print(list_ex_headers)

['LegacyFiles/All_txt/txt_header_examples//Al RRG_030916.txt', 'LegacyFiles/All_txt/txt_header_examples//AL RRG_060403a.txt', 'LegacyFiles/All_txt/txt_header_examples//AL_RRG_091222 remove first tip.txt']


# Dealing with Headers
Next we create a function for working with the messsy headers created by the text files.  There are a few different ones to worry about, we have to manage for each one, while preserving inprotant information, like file name and serial numbers.

In [7]:
def header_fixer(path = "."):
    """
    :param path: path to the txt/csv file
    :return: a dataframe with a fixed header
    """
    csv = pd.read_csv(path, sep="\t") #read in the csv just to check its structure
    
    if csv.shape[1] < 2:  #because files with SN have a multiline header this will filter those files out
        serial_Num = re.findall("[0-9]{5,6}", csv.columns[0])
        csv = pd.read_csv(path, header=1, sep="\t")
        csv["Serial Number"] = serial_Num[0]
    else:
        csv = pd.read_csv(path, sep="\t")
        csv["Serial Number"] =  None

    return csv


great, now lets test it out on our small sample of txt files we created before

In [8]:

ex_headers_csv_list = []

for path in list_ex_headers:
    csv = header_fixer(path = path)
    csv["FileName"] = path.split(sep="/")[-1]
    ex_headers_csv_list.append(csv) #create a list of test dataframes
    display(csv.head())

,Date Time,Event (Rain Bucket Tip),Serial Number,FileName
0,06/18/03 08:40:17.0,0.00,None,Al RRG_030916.txt
1,09/09/03 14:19:42.0,0.01,None,Al RRG_030916.txt
2,09/09/03 15:43:36.5,0.02,None,Al RRG_030916.txt
3,09/09/03 16:33:22.0,0.03,None,Al RRG_030916.txt
4,09/09/03 17:20:45.5,0.04,None,Al RRG_030916.txt


,Date Time,Event (Rain bucket tip),Serial Number,FileName
0,03/26/06 17:58:38.0,0.00,531980,AL RRG_060403a.txt
1,03/27/06 13:23:29.5,0.01,531980,AL RRG_060403a.txt
2,03/27/06 13:37:13.5,0.02,531980,AL RRG_060403a.txt
3,03/27/06 13:47:53.5,0.03,531980,AL RRG_060403a.txt
4,03/27/06 13:53:57.5,0.04,531980,AL RRG_060403a.txt


,Date Time,Event (Events),Serial Number,FileName
0,12/15/09 16:31:03.0,0.000000,None,AL_RRG_091222 remove first tip.txt
1,12/15/09 16:31:24.0,0.010870,None,AL_RRG_091222 remove first tip.txt
2,12/16/09 09:04:57.0,0.021739,None,AL_RRG_091222 remove first tip.txt
3,12/16/09 09:20:12.5,0.032609,None,AL_RRG_091222 remove first tip.txt
4,12/16/09 09:40:41.0,0.043478,None,AL_RRG_091222 remove first tip.txt


# deal with column names
That seems to be working okay, but there are definitely some differences in column names that are going to give us issues later, so lets create a function to deal with those. 

In [9]:
def columnNames_handler(dataframe, column_names = []):
    """takes in a pandas dataframe, and replaces the column names with 
    the ones provided in the column_names list.  if lengths don't match, it retains 
    names exceeding the list, or discards names from the list if there are less 
    columns than the length of the list.

    Args:
        dataframe (pandas dataframe): a pandas dataframe with a cleaned header.
        column_names (list, optional): list of column names (string). Defaults to [].
    """
    df_num_cols = len(dataframe.columns)

    if len(column_names) > df_num_cols: #if there are too many column names given
        column_names = column_names[:(df_num_cols)] 
    elif len(column_names) < df_num_cols: #if there are too few column names given
        column_names = column_names + dataframe.columns[len(column_names):]
    
    dataframe.columns = column_names

    return dataframe



and a test

In [10]:
index = 0

column_names = ["DateTime","BucketTip_Cumm","Serial Number","FileName"]
for df in ex_headers_csv_list:
    ex_headers_csv_list[index] = columnNames_handler(dataframe=df, column_names=column_names)
    print(df.columns)
    index += 1

Index(['DateTime', 'BucketTip_Cumm', 'Serial Number', 'FileName'], dtype='object')
Index(['DateTime', 'BucketTip_Cumm', 'Serial Number', 'FileName'], dtype='object')
Index(['DateTime', 'BucketTip_Cumm', 'Serial Number', 'FileName'], dtype='object')


# deal with cummulative values
As you can see, individual bucket tips were summed up progressively as more tips were recorded.  This is helpful for finding the amount of total rainfall for the length of time between log pulls from precipitation monitors, but will make no sense when combined with other datasets.  There are a couple options for dealing with this:

1. just make one record for each timespan between log pulls in the final dataset
2. solve for each individual bucket tip and make them non-cummulative 

option 2 will allow us to preserve the data in its most granular form, so that future analyists can aggrigate the data as they see fit, so that is what we will work towards.
https://stackoverflow.com/questions/27581942/how-can-i-get-back-real-value-from-cumulative-sum-in-pandas

In [11]:
ex_headers_csv_list[0]["BucketTip_Cumm"] #check accumulation of values

0     0.00
1     0.01
2     0.02
3     0.03
4     0.04
5     0.05
6     0.06
7     0.07
8     0.08
9     0.09
10    0.09
Name: BucketTip_Cumm, dtype: float64

In [12]:
def cumm_to_individual(dataframe, column_index_or_name, indiv_column_name):
    """given a column with cummulative values, add a column with individual non-cummulative records

    Args:
        dataframe (pandas df): _description_
        column_index_or_name (int or str): _description_
        column_name (str): _description_
    """
    if type(column_index_or_name) == int:
        col = dataframe.columns[column_index_or_name]
    else:
        col = column_index_or_name
    
    dataframe[indiv_column_name] = dataframe[col].diff().fillna(dataframe[col].iloc[0])
    return dataframe

cumm_to_individual(dataframe=ex_headers_csv_list[0], column_index_or_name="BucketTip_Cumm", indiv_column_name="precip_individual")["precip_individual"]

0     0.00
1     0.01
2     0.01
3     0.01
4     0.01
5     0.01
6     0.01
7     0.01
8     0.01
9     0.01
10    0.00
Name: precip_individual, dtype: float64

looks much better, now just iterate over all the datasets to give them all that value.

# list all files
Know we need to make a list of all the files we want to try to aggregate.

In [20]:
def search_filetype(path = ".", extentension = ""):
    """
    :param path: string, where to search
    :param extentension: string, what files to look for
    :return: list, paths to all files with a matching extension
    """
    paths = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename[-len(extentension):] == extentension:
                paths.append(dirpath + "/" + filename)
    return paths

#list of all text files in the legacy files directory
list_text_paths = search_filetype(path = "LegacyFiles/", extentension=".txt") 

'C:\\Users\\samer\\OneDrive - Cal Poly\\Classes\\SPR_Fellowship\\SPR_Fellowship'